In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import dask
import dask.dataframe as dd
import time
import os
from IPython.display import display
import ast

# import sweetviz as sv


# print(dask.__version__)

In [37]:
# Convert JSON to parquet files

# Paths to save the Parquet files
# parquet_dir = "../../data/processed/"

# # 1. 
# # Convert pandas DataFrames to Parquet
# data_tip = pd.read_json(f"../../data/raw/yelp_academic_dataset_tip.json", lines=True)
# data_tip.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_tip.parquet"))

# # Convert Dask DataFrames to Parquet
# data_businesses = dd.read_json(f"../../data/raw/yelp_academic_dataset_business.json", lines=True)
# data_businesses.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_business.parquet"))

# data_checkin = dd.read_json(f"../../data/raw/yelp_academic_dataset_checkin.json", lines=True)
# data_checkin.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_checkin.parquet"))

# data_user = dd.read_json(f"../../data/raw/yelp_academic_dataset_user.json", lines=True)
# data_user.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_user.parquet"))

# # Read the large JSON file with Dask in chunks and convert to Parquet
# data_review = dd.read_json(f"../../data/raw/yelp_academic_dataset_review.json", lines=True, blocksize="64MB")
# data_review.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_review.parquet"))



# Read and save sample data for larger files
# data_user = dd.read_json('../../data/raw/yelp_academic_dataset_user.json', lines=True, blocksize=40000000)  # 1MB blocksize
# data_user_sample = data_user.head(5000)  # Adjust the number of rows to read
# data_user_sample.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_review_user.parquet"))

# # Review Data
# data_review = dd.read_json(f"../../data/raw/yelp_academic_dataset_review.json", lines=True, blocksize=40000000)
# data_review = data_review.head(5000)
# data_review.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_review_sample.parquet"))











In [10]:
import dask.dataframe as dd

# Load data and set indices
data_businesses = dd.read_parquet("../../data/processed/yelp_academic_dataset_business.parquet")
data_businesses = data_businesses.set_index('business_id')

data_checkin = dd.read_parquet("../../data/processed/yelp_academic_dataset_checkin.parquet")
data_checkin = data_checkin.set_index('business_id')

data_tip = dd.read_parquet("../../data/processed/yelp_academic_dataset_tip.parquet")

data_review_sample = dd.read_parquet("../../data/processed/yelp_academic_dataset_review_sample.parquet")
data_review_sample = data_review_sample.set_index('review_id')

data_user_sample = dd.read_parquet("../../data/processed/yelp_academic_dataset_user_sample.parquet")
data_user_sample = data_user_sample.set_index('user_id')

# Compute to view the result
bus_df = data_businesses.compute() # bus_id = idx
checkin_df = data_checkin.compute() # bus_id = idx
tip_df = data_tip.compute() 
review_df = data_review_sample.compute()
user_df = data_user_sample.compute()

# Ensure index names are set correctly in Pandas DataFrames
tip_df.index.name = 'tip_num'

df_dict = {
    'bus_df': bus_df,
    'checkin_df': checkin_df,
    'tip_df': tip_df,
    'review_df': review_df,
    'user_df': user_df
}


In [ ]:

# Name , DF pairs
df_dict = {
    'bus_df': bus_df,
    'checkin_df': checkin_df,
    'tip_df': tip_df,
    'review_df': review_df,
    'user_df': user_df
}

# Define the processing instructions
processes = {
    "bus_df": {
        "bool": ["is_open"],  # Convert 0 or 1 to booleans
        "dict": ["attributes", "hours"],  # Convert long strings to dictionaries
        "lists": ["categories"]  # Convert strings to lists
    },
    "checkin_df": {
        "date_time_list": ["date"]  # Split string by ', ' into list, then convert to datetime
    },
    "tip_df": {
        "date_time": ["date"],  # Convert string to datetime
        "string": ["text"]  # Convert to lowercase
    },
    "user_df": {
        "date_time": ["yelping_since"],  # Convert string to datetime
        "dict": ["attributes", "hours"],  # Convert long strings to dictionaries
        "lists": ["elite", "friends"]  # Convert strings to lists
    },
    "review_df": {
        "date_time": ["date"],  # Convert string to datetime
        "string": ["text"]  # Convert to lowercase
    }
}

for df_name in df_dict:
    df = df_dict[df_name]  # Access the dataframe

    # Get processing instructions for the respective dataframe
    instructions = processes[df_name]

    for dtype, cols in instructions.items():
        if dtype == "bool":
            for col in cols:
                df[col] = df[col].astype(bool)  # Convert 0 or 1 to booleans
        
        elif dtype == "date_time":
            for col in cols:
                df[col] = dd.to_datetime(df[col], errors='coerce')  # Convert string to datetime
        
        elif dtype == "date_time_list":
            for col in cols:
                df[col] = df[col].str.split(', ').apply(
                    lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S", errors='coerce') 
                    if pd.notna(x) 
                    else [], 
                    meta=('x', 'datetime64[ns]')
                )  # Convert strings to lists of datetime
        
        elif dtype == "dict":
            for col in cols:
                df[col] = df[col].apply(
                    lambda x: ast.literal_eval(x) if pd.notna(x) else {}, 
                    meta=('x', 'object')
                )  # Convert string to dictionary
        
        elif dtype == "lists":
            for col in cols:
                df[col] = df[col].apply(
                    lambda x: x.split(', ') if pd.notna(x) else [], 
                    meta=('x', 'object')
                )  # Convert string to list
        
        elif dtype == "string":
            for col in cols:
                df[col] = df[col].str.lower()  # Convert to lowercase

    df_dict[df_name] = df  # Update the dictionary with the processed DataFrame
